### Final Entity Matching

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance

def load_df(filename):
    return pd.read_csv(f'./datasets/{filename}.csv')

layout1 = load_df('ABC_layout_1')
layout2 = load_df('PQR_layout_2')
layout3 = load_df('layout_3_voters')
layout4 = load_df('KLM_layout_4')
layout5 = load_df('layout_5_license')

layout1 = layout1.rename(columns={"First Name": "Name", "Father Name": "Father_Name", "Permanent_Adress":"Permanent_Address"})
layout2 = layout2.rename(columns = {"Customer_ID": "Mobile Number"})
layout3 = layout3.rename(columns={"votersName": "Name", "votersFatherName": "Father_Name", "votersMotherName": "Mother Name", " Gender": "Gender", "Permanent_Adress":"Permanent_Address"})
layout4 = layout4.rename(columns={"Father Name": "Father_Name"})

def sanitize(df):
    return df.map(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else '' if pd.isna(x) else x)

layouts = [layout1, layout2, layout3, layout4, layout5]
layout_copies = [layout.copy() for layout in layouts]

for i in range(len(layout_copies)):
    layout_copies[i] = sanitize(layout_copies[i])

def create_soup(df, df_, soup, soup_name):
    df[soup_name] = df_[soup].apply(lambda x: ' '.join(x.values.astype(str)).lower(), axis=1)

soup = ['Name', 'Date of Birth', 'Father_Name']

for i, j, k, in zip(layouts, layout_copies, range(len(layouts))):
    create_soup(i, j, soup, f"soup{k+1}")

def combine_layouts(A, B, soup_A, soup_B, metric = 'cosine', threshold=0.3):
    if metric == 'cosine':
        tfidf = TfidfVectorizer(stop_words='english')
        
        combined_soup = pd.concat([A[soup_A], B[soup_B]], ignore_index=True)
        tfidf.fit(combined_soup)
        
        tfidf_matrix_A = tfidf.transform(A[soup_A])
        tfidf_matrix_B = tfidf.transform(B[soup_B])
        
        similarity = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)
        similarity_df = pd.DataFrame(similarity, index=A.index, columns=B.index)

        max_idx_row = similarity_df.idxmax(axis=1)
        similarity_mask = similarity_df.max(axis=1) > threshold
        
    elif metric == 'levenshtein':
        distance_matrix = pd.DataFrame(np.zeros((len(A), len(B))), index=A.index, columns=B.index)

        for i in A.index:
            for j in B.index:
                distance_matrix.loc[i, j] = distance(A.loc[i, soup_A], B.loc[j, soup_B])

        min_idx_row = distance_matrix.idxmin(axis=1)
        min_distance = distance_matrix.min(axis=1)

        similarity_mask = min_distance <= threshold
    
    # Initialize the combined DataFrame with A, ensuring all columns from both DataFrames
    combined_columns = list(set(A.columns) | set(B.columns))
    combined_data = pd.DataFrame(columns=combined_columns)
    
    # Merge the similar rows 
    for idx_A in A.index:
        if similarity_mask[idx_A]:
            idx_B = max_idx_row[idx_A]
            combined_row = A.loc[idx_A].combine_first(B.loc[idx_B])
        else:
            combined_row = A.loc[idx_A]
        combined_data = pd.concat([combined_data, combined_row.to_frame().T], ignore_index=True)
    
    # Append non-similar rows from B to A
    new_records = B.loc[~B.index.isin(max_idx_row[similarity_mask].values)]
    result = pd.concat([combined_data, new_records], ignore_index=True)
    result.drop(columns=soup_B, inplace=True)
    return result


result_12 = combine_layouts(layout1, layout2, 'soup1', 'soup2')
result_123 = combine_layouts(result_12, layout3, 'soup1', 'soup3')
result_1234 = combine_layouts(result_123, layout4, 'soup1', 'soup4')
final_result = combine_layouts(result_1234, layout5, 'soup1', 'soup5')
final_result.drop(columns='soup1', inplace=True)
final_result

,SC Number,Temporary_Address,Father_Name,Mobile Number,Blood Group,PAN_Number,Customer Code,Permanent_Address,License Number,Mother Name,Name,Gender,votersID,Citizenship Number,SpouseName,National Id,Date of Birth,votersAge,Customer ID
0,001.01.01,"Gongabu, Kathmandu, Nepal",Ram Bahadur Thapa,1234567890,AB+,ABCDE1234F,21216874,"Baluwatar, Kathmandu, Nepal",15-05-58353205,Laxmi Thapa,Ram Thapa,Male,11116874,624-93227-32431/660086,Sita Thapa,AB123C,1990-01-01,45,3245
1,001.01.02,"New Road, Pokhara, Nepal",Hari Prasad Shrestha,2345678901,AB-,FGHIJ5678K,22359363,"Lakeside, Pokhara, Nepal",21-08-00435579,Radha Sharma,Sita Shrestha,Female,22259363,747-42087-31417/584714,Ravi Sharma,DE456F,1991-02-02,38,3246
2,001.01.03,"Pulchowk, Lalitpur, Nepal",Gopal Krishna Gurung,3456789012,B-,LMNOP9012L,33485241,"Chitwan National Park, Chitwan, Nepal",93-12-35351480,Gita Adhikari,Hari Gurung,Male,33385241,389-45382-93886/821590,Maya Adhikari,GH789I,1992-03-03,52,3247
3,001.01.04,"Bagbazar, Kathmandu, Nepal",Shyam Lal Tamang,4567890123,A-,QRSTU3456M,45475489,"Biratnagar, Morang, Nepal",65-03-68139881,Mina Rai,Gita Tamang,Female,44475489,571-38785-99733/440035,Surya Rai,JK012L,1993-04-04,30,3248
4,001.01.05,"Balkumari, Lalitpur, Nepal",Krishna Raj Lama,5678901234,A+,VWXYZ7890N,56562139,"Bharatpur, Chitwan, Nepal",14-11-40056582,Kalpana Karki,Mohan Lama,Male,55562139,864-17331-40021/961722,Sarita Karki,MN345O,1994-05-05,27,3249
5,001.01.06,"Chabahil, Kathmandu, Nepal",Narayan Kumar Magar,6789012345,B-,ABCD1234PQ,67698214,"Butwal, Rupandehi, Nepal",82-09-81734599,Nima Gurung,Radha Magar,Female,66698214,033-51347-62581/380746,Pemba Gurung,PQ678R,1995-06-06,41,3250
6,001.01.07,"Kumaripati, Lalitpur, Nepal",Govinda Bahadur Rai,7890123456,AB+,EFGH5678RS,78714635,"Hetauda, Makwanpur, Nepal",26-03-50185868,Saru Shrestha,Krishna Rai,Male,77714635,068-37653-84341/852787,Laxmi Shrestha,ST901U,1996-07-07,36,3251
7,001.01.08,"New Baneshwor, Kathmandu, Nepal",Shiva Narayan Sherpa,8901234567,A-,TUVW9012XY,89847326,"Janakpur, Dhanusa, Nepal",98-03-72394228,Nanu Maharjan,Sarita Sherpa,Female,88847326,323-37869-95909/623481,Raj Maharjan,VW234X,1997-08-08,50,3252
8,001.01.09,"Boudha, Kathmandu, Nepal",Bhagirath Bahadur Karki,9012345678,B-,ZABC3456DE,91953421,"Nepalgunj, Banke, Nepal",68-01-80388982,Lhamu Tamang,Bikash Karki,Male,99953421,406-94259-70142/249811,Pema Tamang,YZ567A,1998-09-09,43,3253
9,001.01.10,"Kalanki, Kathmandu, Nepal",Surya Bahadur Bhandari,123456789,A-,FGHI7890JK,10203847,"Dharan, Sunsari, Nepal",42-08-02614125,Lila KC,Nisha Bhandari,Female,10103847,272-28301-42325/881177,Manish KC,BC890D,1999-10-10,34,3254
